In [ ]:
# function ClickConnect(){
# console.log("Working"); 
# document.querySelector("colab-toolbar-button").click() 
# }setInterval(ClickConnect, 1800000)

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/')

%cd '/content/lyft-motion-prediction-autonomous-vehicles/'
!pwd

/content
/content/lyft-motion-prediction-autonomous-vehicles


In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip 
!pip install kaggle==1.5.6

!mkdir -p ~/.kaggle
!cp ../kaggle.json ~/.kaggle/

!kaggle competitions download -c lyft-motion-prediction-autonomous-vehicles
!unzip lyft-motion-prediction-autonomous-vehicles.zip

In [ ]:
# import os
# os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/', exist_ok=True)
# os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/')
%cd '/content/lyft-motion-prediction-autonomous-vehicles/'

!pip install --target=/content/lyft-motion-prediction-autonomous-vehicles pymap3d==2.1.0 -q
!pip install --target=/content/lyft-motion-prediction-autonomous-vehicles strictyaml -q
!pip install --target=/content/lyft-motion-prediction-autonomous-vehicles protobuf==3.12.2 -q
!pip install --target=/content/lyft-motion-prediction-autonomous-vehicles transforms3d -q
!pip install --target=/content/lyft-motion-prediction-autonomous-vehicles zarr -q
!pip install --target=/content/lyft-motion-prediction-autonomous-vehicles ptable -q
!pip install --no-dependencies --target=/content/lyft-motion-prediction-autonomous-vehicles l5kit==1.1.0 --upgrade -q

# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl
VERSION = "nightly"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

!pip install pytorch-lightning==1.0.4
!pip install adamp

In [ ]:
# !pip install l5kit
# !pip install adamp
# !pip install pytorch-lightning

In [ ]:
INPUT_DIR = '/content/lyft-motion-prediction-autonomous-vehicles/'

In [ ]:
from __future__ import print_function, division, absolute_import
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as plt

import os
import random
import time


from collections import OrderedDict
import math

import torch.nn as nn
from torch.utils import model_zoo
from adamp import AdamP
from pytorch_lightning import Trainer
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp

import gc

# import warnings
# warnings.filterwarnings("ignore")

# l5kit.__version__

# def set_seed(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)
#     torch.manual_seed(seed)
#     # torch.cuda.manual_seed(seed)
    
# set_seed(42)

In [ ]:
cfg = {
    'format_version': 4,
    'data_path': "/content/lyft-motion-prediction-autonomous-vehicles",
    'model_params': {
        'model_architecture': 'resnet18',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "resnest50+267+adamp+0.5+10+1e-4+yaw_head3",
        # 'model_name': "lr_finder_test",
        'lr': 1e-4,
        # 'weight_path': "/content/lyft-motion-prediction-autonomous-vehicles/pytorch_lightning-models-v21.ckpt",
        # 'weight_path': "./model_resnet34_output_0.pth",
        'weight_path': None,
        'train': True,
        'predict': True,
    },

    'raster_params': {
        'raster_size': [267, 267],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False
    },

    'train_data_loader': {
        'key': 'scenes/sample.zarr',
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 8
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 8
    },
    'val_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 8
    },

    'train_params': {
        'max_num_steps': 101,
        'checkpoint_every_n_steps': 20,
    },
}

In [ ]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

In [ ]:
class LyftMultiModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.lr = cfg["model_params"]["lr"]
        self.num_modes = 3
        model_name = cfg["model_params"]["model_name"]


        architecture = cfg["model_params"]["model_architecture"]
        # backbone = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        backbone = eval(architecture)(pretrained=True, progress=True)
        
        self.backbone = backbone

        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels
        # print(self.backbone.conv1[0])
        if architecture == "resnest50" or architecture == "resnest101":
          self.backbone.conv1[0] = nn.Conv2d(
              num_in_channels,
              # self.backbone.conv1[0].out_channels,
              32,
              # kernel_size=self.backbone.conv1[0].kernel_size,
              kernel_size=(3,3),
              # stride=self.backbone.conv1[0].stride,
              stride=(2,2),
              # padding=self.backbone.conv1[0].padding,
              padding=(1,1),
              bias=False,
          )
        else:
            self.backbone.conv1 = nn.Conv2d(
                num_in_channels,
                self.backbone.conv1.out_channels,
                kernel_size=self.backbone.conv1.kernel_size,
                stride=self.backbone.conv1.stride,
                padding=self.backbone.conv1.padding,
                bias=False,
            )

        if architecture == "resnet50" or architecture == "resnext50" or architecture == "resnest50" or architecture == "resnext101":
            backbone_out_features = 2048
        else:
            backbone_out_features = 512


        self.future_len = cfg["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len


        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
            # nn.Linear(in_features=4096, out_features=1024),
            # nn.Linear(in_features=1024, out_features=256)
        )


        self.num_preds = num_targets * self.num_modes


        self.x_preds = nn.Linear(4096, out_features=self.num_preds)
        self.x_modes = nn.Linear(4096, out_features=self.num_modes)

        


    def forward(self, batch):
        x = self.backbone.conv1(batch)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        # yaws = torch.abs(yaws)
        # yaws_norm = torch.div(yaw+1e-10, 7.079)
        # for i in range(yaws.shape[0]):
          # x[i] = torch.mul(x[i], yaws_norm[i])
        x = self.head(x)

        preds = self.x_preds(x)
        modes = self.x_modes(x)
        return preds, modes

In [ ]:
def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    preds, modes = model(inputs)
    ############################################################################
    bs, _ = preds.shape
    preds = preds.view(bs, 3, 50, 2)
    assert modes.shape == (bs, 3)
    confidences = torch.softmax(modes, dim=1)
    ############################################################################
    loss = criterion(targets, preds, confidences, target_availabilities)
    return loss, preds, confidences

In [ ]:
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

In [ ]:
def simple_map_fn(index, flags):
  torch.manual_seed(1234)

  device = xm.xla_device()  

  # if xm.is_master_ordinal():
  t = torch.randn((2, 2), device=device)
  # out = str(t)

  # if xm.is_master_ordinal():
  #   print(out)
  
  print("Process", index ,"is using", xm.xla_real_devices([str(device)])[0])

  xm.rendezvous('init')

flags = {}
xmp.spawn(simple_map_fn, args=(flags,), nprocs=8, start_method='fork')

In [ ]:
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch_xla.distributed.parallel_loader as pl
import time


def map_fn(index, flags):
  torch.manual_seed(flags['seed'])

  device = xm.xla_device()

  if not xm.is_master_ordinal():
    xm.rendezvous('download_only_once')

  test_cfg = cfg["test_data_loader"]
  rasterizer = build_rasterizer(cfg, dm)
  test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
  test_dataset = AgentDataset(cfg, test_zarr, rasterizer)

  if xm.is_master_ordinal():
    xm.rendezvous('download_only_once')

  test_sampler = torch.utils.data.distributed.DistributedSampler(
    test_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True
  )


  test_loader = DataLoader(
      test_dataset, 
      batch_size=flags['batch_size'],  
      sampler=test_sampler, 
      num_workers=flags['num_workers'],
      drop_last=True)
  

  net = LyftMultiModel().to(device)
  loss_fn = pytorch_neg_multi_log_likelihood_batch
  optimizer = AdamP(net.parameters(), lr=0.0001, betas=(0.9, 0.999), weight_decay=1e-2)


  num_modes = 3
  future_len = 50
  loader_len = len(train_loader)
  
  # for epoch in range(flags['num_epochs']):


  net.eval()
  with torch.no_grad():
    para_val_loader = pl.ParallelLoader(val_loader, [device]).per_device_loader(device)
    for batch_num, batch in enumerate(para_val_loader):
      # loss, _, _ = forward(batch, device, net)
      inputs = batch["image"].to(device)
      target_availabilities = batch["target_availabilities"].to(device)
      targets = batch["target_positions"].to(device)
      # Forward pass
      preds, modes = net(inputs)
      ############################################################################
      bs, _ = preds.shape
      preds = preds.view(bs, 3, 50, 2)
      assert modes.shape == (bs, 3)
      confidences = torch.softmax(modes, dim=1)
      ############################################################################
      loss = loss_fn(targets, preds, confidences, target_availabilities)
      # images = batch["image"]
      # yaws = batch["yaw"]
      # target_availabilities = batch["target_availabilities"]
      # targets = batch["target_positions"]

      # pred, confidences = net(images, yaws)
      # bs, _ = pred.shape
      # preds = pred.view(bs, num_modes, future_len, 2)
      # assert confidences.shape == (bs, num_modes)
      # confidences = torch.softmax(confidences, dim=1)
      # loss = loss_fn(targets, preds, confidences, target_availabilities)

      losses.append(loss)

    times.append(elapsed_train_time)
    print("Process", index, "loss", torch.mean(losses_val))

In [ ]:
flags['batch_size'] = 64
flags['num_workers'] = 0
flags['num_epochs'] = 1
flags['seed'] = 1234
flags['log_steps'] = 10

avg_loss_set = []
iterations = []
time_set = []
metrics = []

losses = []
xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')
# for epoch in range(flags['num_epochs']):
#   losses = []


#   iterations.append(epoch + 1)
#   avg_loss_set.append(torch.mena(losses))
#   time_set.append(torch.mean(times))
#   results = pd.DataFrame({'iterations': iterations, 'losses': avg_loss_set, 'elapsed_time (mins)': time_set})
#   results.to_csv(f"/content/drive/My Drive/kaggle/lyft-motion-prediction-autonomous-vehicle/{cfg['model_params']['model_name']}epoch{epoch}.csv", index = False)

In [ ]:
# write_pred_csv(f'/content/drive/My Drive/kaggle/lyft-motion-prediction-autonomous-vehicle/{cfg["model_params"]["model_name"]}_submission.csv',
#            timestamps=np.concatenate(timestamps),
#            track_ids=np.concatenate(agent_ids),
#            coords=np.concatenate(future_coords_offsets_pd),
#            confs = np.concatenate(confidences_list)
#           )